In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pyecharts import HeatMap
df = pd.read_csv('./data/HE_2013_M25_59_1_10.csv', delimiter=';', parse_dates=['date_time'])  # 59条链路，1-10月原始数据
df = df.drop(['Local_Date', 'Local_Time', 'Day_Type_ID'], axis=1)  # 删除部分无关字段
df.columns = ['link_ID', 'date', 'length', 'flow', 'travel_time', 'speed', 'q', 'travel_time_log']  # 重命名
print df.count()
df.head()

link_ID            1721856
date               1721856
length             1624946
flow               1624944
travel_time        1624946
speed              1624946
q                  1624946
travel_time_log    1624946
dtype: int64


,link_ID,date,length,flow,travel_time,speed,q,travel_time_log
0,LM329,2013-01-01 00:00:00,5260.000229,45.0,172.02,110.08,3.0,5.153407
1,LM329,2013-01-01 00:15:00,5260.000229,54.0,177.29,106.81,3.0,5.183411
2,LM329,2013-01-01 00:30:00,5260.000229,112.0,176.77,107.12,3.0,5.180491
3,LM329,2013-01-01 00:45:00,5260.000229,102.0,176.00,107.59,3.0,5.176150
4,LM329,2013-01-01 01:00:00,5260.000229,155.0,176.63,107.21,1.0,5.179703


## 填充NAN
- 方案一： 往前找可用数据点
- 方案二： 前后时间片的均值
- 方案三： 前面四个时间点的均值
- 方案四： 该时间片的历史均值
- 方案五： 特殊值
- 方案六： xboost预测
- 方案七： 融合

使用方案
 - step1：往前找可用数据点，做多三个点
 - step2：前后四个时间点均值

In [52]:
print 'The number of NAN is %d' % (df.count()[0] - df.count()[7])

The number of NAN is 96910


In [43]:
df['travel_time_log_t-1'] = df.shift(-1)['travel_time_log']

In [56]:
for i in range(100):
    df.travel_time_log = df.travel_time_log.fillna(df.shift(-1)['travel_time_log'])
    print 'The number of NAN is %d after shift %d' % ((df.count()[0] - df.count()[7]), i)

The number of NAN is 75706 after shift 0
The number of NAN is 75297 after shift 1
The number of NAN is 74888 after shift 2
The number of NAN is 74479 after shift 3
The number of NAN is 74070 after shift 4
The number of NAN is 73661 after shift 5
The number of NAN is 73252 after shift 6
The number of NAN is 72843 after shift 7
The number of NAN is 72434 after shift 8
The number of NAN is 72025 after shift 9
The number of NAN is 71616 after shift 10
The number of NAN is 71207 after shift 11
The number of NAN is 70798 after shift 12
The number of NAN is 70389 after shift 13
The number of NAN is 69980 after shift 14
The number of NAN is 69571 after shift 15
The number of NAN is 69162 after shift 16
The number of NAN is 68753 after shift 17
The number of NAN is 68344 after shift 18
The number of NAN is 67935 after shift 19
The number of NAN is 67526 after shift 20
The number of NAN is 67117 after shift 21
The number of NAN is 66708 after shift 22
The number of NAN is 66299 after shift 23
Th

In [46]:
df

,link_ID,date,length,flow,travel_time,speed,q,travel_time_log,travel_time_log_t-1
0,LM329,2013-01-01 00:00:00,5260.000229,45.0,172.02,110.08,3.0,5.153407,5.183411
1,LM329,2013-01-01 00:15:00,5260.000229,54.0,177.29,106.81,3.0,5.183411,5.180491
2,LM329,2013-01-01 00:30:00,5260.000229,112.0,176.77,107.12,3.0,5.180491,5.176150
3,LM329,2013-01-01 00:45:00,5260.000229,102.0,176.00,107.59,3.0,5.176150,5.179703
4,LM329,2013-01-01 01:00:00,5260.000229,155.0,176.63,107.21,1.0,5.179703,5.205709
5,LM329,2013-01-01 01:15:00,5260.000229,154.0,181.31,104.44,1.0,5.205709,5.180997
6,LM329,2013-01-01 01:30:00,5260.000229,175.0,176.86,107.07,1.0,5.180997,5.182457
7,LM329,2013-01-01 01:45:00,5260.000229,144.0,177.12,106.91,1.0,5.182457,5.208010
8,LM329,2013-01-01 02:00:00,5260.000229,151.0,181.73,104.20,1.0,5.208010,5.183860
9,LM329,2013-01-01 02:15:00,5260.000229,111.0,177.37,106.76,3.0,5.183860,5.162956


In [47]:
print 'The number of NAN is %d' % (df.count()[0] - df.count()[7])

The number of NAN is 96362


In [50]:
df[df.travel_time_log.isnull() == True]

,link_ID,date,length,flow,travel_time,speed,q,travel_time_log,travel_time_log_t-1
1056,LM329,2013-01-12 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1057,LM329,2013-01-12 00:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1058,LM329,2013-01-12 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1059,LM329,2013-01-12 00:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1060,LM329,2013-01-12 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1061,LM329,2013-01-12 01:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1062,LM329,2013-01-12 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1063,LM329,2013-01-12 01:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1064,LM329,2013-01-12 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1065,LM329,2013-01-12 02:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
